In [2]:
import requests
from bs4 import BeautifulSoup
from pandas_datareader import data as wb
import pandas as pd
import pandas_ta as ta
import time
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from pylab import mpl, plt

In [24]:
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'

In [25]:
# coleta de nome do papel s&p500
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
agent = {'User-Agent':'Mozzila/5.0'}
resposta = requests.get(url, headers=agent)
soup = BeautifulSoup(resposta.text, 'lxml')
tabela = soup.find_all('table')[0]
df = pd.read_html(str(tabela), decimal=',', thousands='.')[0]
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [26]:
df['Symbol'].head()

0     MMM
1     AOS
2     ABT
3    ABBV
4    ABMD
Name: Symbol, dtype: object

In [27]:
Symbols = df['Symbol']
Symbols

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, Length: 505, dtype: object

In [28]:
ativos = [tick for tick in Symbols]
ativos

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [29]:
def get_ohlc(ativo, start='2020-1-1'):
    df = wb.DataReader(ativo, data_source='yahoo', start='2020-1-1') 
    return df

In [30]:
# calculo percentil para determinar intervalo de confiança

def percentil(data, up=90, down=10):
    x = data
    x = x.dropna()
    array_x = np.array(x)
    percentil_up = np.percentile(array_x, up)
    percentil_down = np.percentile(array_x, down)
    
    return [percentil_up, percentil_down]

In [31]:
def caroBarato(tick_ativo, up, down, start):

    df = get_ohlc(tick_ativo, start)

    # criação médias
    df['M20'] = ta.sma(df['Adj Close'], length=20)
    df['M50'] = ta.sma(df['Adj Close'], length=50)
    df['M200'] = ta.sma(df['Adj Close'], length=200)

    #criação distancia média
    df['Dist20'] = df['Adj Close'] - df['M20'] 
    df['Dist50'] = df['Adj Close'] - df['M50']
    df['Dist200'] = df['Adj Close'] - df['M200']

    # calculo percentil das distancia da média 
    # return percentil_up, percentil_down
    per_m20 = percentil(df['Dist20'], up, down)
    per_m50 = percentil(df['Dist50'], up, down)
    per_m200 = percentil(df['Dist200'], up, down)

    # transformação em um array
    
    # variavel temporaria
    valor = 0

    # logica
    # logica valor = +1 +2 +4 or -1 -2 -4 para diferenciar qual média aponta distancia
    if df['Dist20'][-1] > per_m20[0]:
        valor += 1
    
    if df['Dist50'][-1] > per_m50[0]:
        valor += 2
    
    if df['Dist200'][-1] > per_m200[0]:
        valor += 4
    
    if df['Dist20'][-1] < per_m20[1]:
        valor -= 1
    
    if df['Dist50'][-1] < per_m50[1]:
        valor -= 2
    
    if df['Dist200'][-1] < per_m200[1]:
        valor -= 4

    return valor

In [32]:
caroBarato('PG', 95, 5, start='2020-1-1')

0

In [33]:
dicionarioCarobarato = {}
for ativo in ativos:
    try:
        dicionarioCarobarato[ativo] = caroBarato(ativo, 95, 5, start='2020-1-1')
        print(f'COLETA DE {ativo}')
    except:
        print(f'{ativo} SEM COLETA')
        #dicionarioCarobarato[ativo] = 'NAN'
dicionarioCarobarato = pd.DataFrame.from_dict(dicionarioCarobarato, orient='index').reset_index()

COLETA DE MMM
COLETA DE AOS
COLETA DE ABT
COLETA DE ABBV
COLETA DE ABMD
COLETA DE ACN
COLETA DE ATVI
COLETA DE ADM
COLETA DE ADBE
COLETA DE ADP
COLETA DE AAP
COLETA DE AES
COLETA DE AFL
COLETA DE A
COLETA DE AIG
COLETA DE APD
COLETA DE AKAM
COLETA DE ALK
COLETA DE ALB
COLETA DE ARE
COLETA DE ALGN
COLETA DE ALLE
COLETA DE LNT
COLETA DE ALL
COLETA DE GOOGL
COLETA DE GOOG
COLETA DE MO
COLETA DE AMZN
COLETA DE AMCR
COLETA DE AMD
COLETA DE AEE
COLETA DE AAL
COLETA DE AEP
COLETA DE AXP
COLETA DE AMT
COLETA DE AWK
COLETA DE AMP
COLETA DE ABC
COLETA DE AME
COLETA DE AMGN
COLETA DE APH
COLETA DE ADI
COLETA DE ANSS
COLETA DE ANTM
COLETA DE AON
COLETA DE APA
COLETA DE AAPL
COLETA DE AMAT
COLETA DE APTV
COLETA DE ANET
COLETA DE AJG
COLETA DE AIZ
COLETA DE T
COLETA DE ATO
COLETA DE ADSK
COLETA DE AZO
COLETA DE AVB
COLETA DE AVY
COLETA DE BKR
COLETA DE BLL
COLETA DE BAC
COLETA DE BBWI
COLETA DE BAX
COLETA DE BDX
BRK.B SEM COLETA
COLETA DE BBY
COLETA DE BIO
COLETA DE TECH
COLETA DE BIIB
COLETA DE BLK

In [34]:
dicionarioCarobarato.columns = ["Ticker", "Valor"]
dicionarioCarobarato

,Ticker,Valor
0,MMM,-5
1,AOS,-2
2,ABT,0
3,ABBV,4
4,ABMD,-4
...,...,...
489,YUM,-4
490,ZBRA,-4
491,ZBH,-4
492,ZION,0


In [35]:
ativos_caros = dicionarioCarobarato[dicionarioCarobarato['Valor'] > 4]
ativos_caros.sort_values(by='Valor')

,Ticker,Valor
114,CME,5
37,ABC,6
102,CVX,6
122,COP,6
170,EOG,6
220,HAL,6
226,HSY,6
288,LMT,6
305,MCK,6
370,PXD,6


In [36]:
ativos_baratos = dicionarioCarobarato[dicionarioCarobarato['Valor'] < -4]
ativos_baratos.sort_values(by='Valor')

,Ticker,Valor
240,IEX,-7
243,ITW,-7
285,LIN,-7
308,FB,-7
201,FBHS,-7
...,...,...
163,ECL,-6
261,JBHT,-5
224,PEAK,-5
232,HON,-5


In [37]:
# colocar filtro de liquidez
# colocar coleta yahoo finace
# colocar Scatterplot 

In [38]:
# filtro diferente de 0
dicionarioCarobarato = dicionarioCarobarato[dicionarioCarobarato['Valor'] != 0]

In [39]:
dicionarioCarobarato = dicionarioCarobarato.sort_values(by='Valor')

In [40]:
import plotly.express as px
import seaborn as sns

fig = px.scatter(dicionarioCarobarato, x='Ticker', y='Valor',  hover_name='Ticker')
fig.show()

In [41]:
dicionarioCarobarato.to_csv('sp500_caros_baratos.csv')